In [1]:
import numpy as np
import pandas as pd

<b> Загрузка и предобработка исходных данных

In [2]:
data = pd.read_csv("../../Практические материалы/Lab 1. Linear and logistic regression/mnist/mnist_train.csv")
data_test = pd.read_csv("../../Практические материалы/Lab 1. Linear and logistic regression/mnist/mnist_test.csv")

In [3]:
data.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
X = data.iloc[0:2,1:].values
X_test = data_test.iloc[:,1:].values

In [32]:
mean = np.mean(X)
std = np.std(X)

X = (X - mean)/std
X_test = (X_test - mean)/std

In [33]:
m = len(X)
n = 10

Y = np.zeros([m,n])

for i in range(m):
    Y[i,data.label.values[i]] = 1

In [34]:
Y

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [35]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [36]:
def tanh(z):
    return (np.exp(z)-np.exp(-z))/(np.exp(z)+np.exp(-z))

In [37]:
def relu(z):
    return np.max(0,z)

In [38]:
def softmax(z):
    return np.exp(z)/np.sum(np.exp(z), axis = 1).reshape([len(z),1])

In [39]:
def forward(W1,W2,W3,B1,B2,B3,X):
    F1 = sigmoid(np.matmul(X,W1)+B1)
    F2 = sigmoid(np.matmul(F1,W2)+B2)
    F3 = sigmoid(np.matmul(F2,W3)+B3)
    H = softmax(F3)
    #H = F3.copy()
    
    return F1,F2,F3,H

In [45]:
def backpropagation(W1,W2,W3,B1,B2,B3,X,F1,F2,F3,H,Y):
    sigma3 = (H - Y)/(H*(1 - H))    # regression: (H - Y)*H*(1 - H)
    
    d_softmax = np.zeros([10,10])
    for i in range(d_softmax.shape[0]):
        for j in range(d_softmax.shape[1]):
            if i == j:
                d_softmax[i,j] += H[:,i]*(1-H[:,j])
            else:
                d_softmax[i,j] += -H[:,i]*H[:,j]
                #pass
    
    print(d_softmax.shape)
    print(sigma3.shape)
    
    sigma3 = np.matmul(sigma3,d_softmax)
    
    sigma3 = sigma3*F3*(1 - F3)
    
    dW3 = np.matmul(F2.T,sigma3)
    dB3 = np.sum(sigma3, axis = 0)
    sigma2 = np.matmul(sigma3,W3.T)*F2*(1 - F2) # (1-F2**2) (F2>=0)*1 1
    dW2 = np.matmul(F1.T,sigma2)
    dB2 = np.sum(sigma2, axis = 0)
    sigma1 = np.matmul(sigma2,W2.T)*F1*(1 - F1)
    dW1 = np.matmul(X.T,sigma1)
    dB1 = np.sum(sigma1, axis = 0)
    
    return dW1,dW2,dW3,dB1,dB2,dB3

In [43]:
k1 = 100
k2 = 50
k3 = 10
n = 784

np.random.seed(1)
W1 = np.random.randn(n,k1)
W2 = np.random.randn(k1,k2)
W3 = np.random.randn(k2,k3)
B1 = np.zeros([1,k1])
B2 = np.zeros([1,k2])
B3 = np.zeros([1,k3])
#dropout1 = (np.random.rand(n,k1) > 0.3)*1
#dropout2 = (np.random.rand(k1,k2) > 0.3)*1
#dropout3 = (np.random.rand(k2,k3) > 0.3)*1

alpha = 0.00005

for i in range(1000):
    F1,F2,F3,H = forward(W1,W2,W3,B1,B2,B3,X)
    J = -np.sum(Y*np.log(H)+(1-Y)*np.log(1-H))
    if i%10 == 0:
        print(i,J)
    dW1,dW2,dW3,dB1,dB2,dB3 = backpropagation(W1,W2,W3,B1,B2,B3,X,F1,F2,F3,H,Y)
    
    break
    W3 = W3 - alpha*dW3
    W2 = W2 - alpha*dW2
    W1 = W1 - alpha*dW1
    B3 = B3 - alpha*dB3
    B2 = B2 - alpha*dB2
    B1 = B1 - alpha*dB1
    break

0 6.789729941238042
(10, 10)
(2, 10)


In [44]:
_,_,_,H1 = forward(W1,W2,W3,B1,B2,B3,X)
J1 = -np.sum(Y*np.log(H1)+(1-Y)*np.log(1-H1))
W3[4,8] += 0.0000001
_,_,_,H2 = forward(W1,W2,W3,B1,B2,B3,X)
J2 = -np.sum(Y*np.log(H2)+(1-Y)*np.log(1-H2))
W3[4,8] -= 0.0000001

print((J2-J1)/0.0000001)
print(dW3[4,8])

0.0018879120489145862
0.003764177596804249


In [244]:
W3.shape

(50, 10)

In [15]:
_,_,pred = forward(W1,W2,W3,B1,B2,B3,X_test)

In [16]:
pred = np.argmax(pred,axis=1)

In [17]:
np.mean(pred == data_test.label.values)

0.9175

In [18]:
for cls in range(10):
    print(np.sum((data_test.label.values == cls) * (pred == data_test.label.values))/np.sum(data_test.label.values == cls))

0.9693877551020408
0.9770925110132158
0.9118217054263565
0.8841584158415842
0.9134419551934827
0.8733183856502242
0.9405010438413361
0.9153696498054474
0.8747433264887063
0.9038652130822596


In [155]:
from keras.models import Sequential
from keras.layers import Dense, Softmax
from keras import initializers
from keras.optimizers import SGD, Adam

W_init = initializers.RandomNormal(stddev = 1)
B_init = initializers.Zeros()

model = Sequential()

model.add(Dense(100, input_shape=[784], activation='sigmoid',kernel_initializer=W_init,bias_initializer=B_init))
model.add(Dense(50, activation='sigmoid',kernel_initializer=W_init,bias_initializer=B_init))
model.add(Dense(10, activation='sigmoid',kernel_initializer=W_init,bias_initializer=B_init))

optimizer = Adam(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["categorical_crossentropy","accuracy"])

In [156]:
print(model.summary())

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_74 (Dense)             (None, 100)               78500     
_________________________________________________________________
dense_75 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_76 (Dense)             (None, 10)                510       
Total params: 84,060
Trainable params: 84,060
Non-trainable params: 0
_________________________________________________________________
None


In [165]:
for i in range(10):
    model.fit(X, Y, epochs = 1, batch_size=len(X))
    H = model.predict(X)
    J = -np.sum(Y*np.log(H)+(1-Y)*np.log(1-H))
    print(J)

Epoch 1/1
60000/60000 [==============================] - 2s 36us/step - loss: 0.7352 - categorical_crossentropy: 0.7352 - accuracy: 0.7539
121042.75996251479
Epoch 1/1
60000/60000 [==============================] - 2s 31us/step - loss: 0.7104 - categorical_crossentropy: 0.7104 - accuracy: 0.7660
121029.7733215774
Epoch 1/1
60000/60000 [==============================] - 2s 28us/step - loss: 0.6885 - categorical_crossentropy: 0.6885 - accuracy: 0.7765
120606.02262715591
Epoch 1/1
60000/60000 [==============================] - 2s 27us/step - loss: 0.6673 - categorical_crossentropy: 0.6673 - accuracy: 0.7872
119883.96072103534
Epoch 1/1
60000/60000 [==============================] - 2s 30us/step - loss: 0.6458 - categorical_crossentropy: 0.6458 - accuracy: 0.7966
119113.08899408921
Epoch 1/1
60000/60000 [==============================] - 2s 28us/step - loss: 0.6249 - categorical_crossentropy: 0.6249 - accuracy: 0.8044
118515.66588374053
Epoch 1/1
60000/60000 [==============================

In [166]:
H = model.predict(X)
J = -np.sum(Y*np.log(H)+(1-Y)*np.log(1-H))
print(J)
print(J/60000/10)

118366.84861826885
0.19727808103044808


In [167]:
W1_keras = np.array(model.weights[0][:,:])
B1_keras = np.array(model.weights[1][:])
W2_keras = np.array(model.weights[2][:,:])
B2_keras = np.array(model.weights[3][:])
W3_keras = np.array(model.weights[4][:,:])
B3_keras = np.array(model.weights[5][:])

In [168]:
np.corrcoef(B3.flatten(),B3_keras.flatten())

array([[ 1.        , -0.46641235],
       [-0.46641235,  1.        ]])

In [169]:
H = model.predict(X_test)
pred = np.argmax(H,axis=1)
np.mean(pred == data_test.label.values)

0.8331

In [519]:
for cls in range(10):
    print(np.sum((data_test.label.values == cls) * (pred == data_test.label.values))/np.sum(data_test.label.values == cls))

NameError: name 'pred' is not defined

In [520]:
a1

array([[ -7,   2, -15],
       [  2, -18,  16],
       [ 10, -11,  -2]])

In [516]:
A = np.array([[3,6],[-4,2]])
B = np.array([[54],[8]])

In [517]:
X = np.matmul(np.linalg.pinv(A),B)

In [518]:
X

array([[2.],
       [8.]])

In [497]:
X

array([[ -52, -160,   89]])